# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [34]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [25]:
vacationdf= pd.read_csv('City_data.csv')
vacationdf


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,WindSpeed,Country,Date
0,0,Bonfim,3.08,-59.95,88.610,62,85,0.80,BR,1591478468
1,1,Lebu,-37.62,-73.65,53.042,85,100,1.97,CL,1591478468
2,2,Mataura,-46.19,168.86,41.072,86,83,3.20,NZ,1591478469
3,3,Oudtshoorn,-33.59,22.20,59.000,87,0,2.10,ZA,1591478469
4,4,Port Alfred,-33.59,26.89,60.998,94,56,1.34,ZA,1591478469
...,...,...,...,...,...,...,...,...,...,...
657,657,Ølgod,55.81,8.63,51.998,76,40,6.20,DK,1591479079
658,658,Salinópolis,-0.61,-47.36,80.924,80,100,4.46,BR,1591479079
659,659,Nago,26.59,127.98,77.000,94,90,2.10,JP,1591479079
660,660,Sorong,-0.88,131.25,78.044,85,100,1.64,ID,1591479079


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [24]:
locations = vacationdf[["Lat", "Lng"]]
humidity = vacationdf["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=8)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [30]:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#less than 30% cloudiness.
#Humidity less than 30%

ideal_df=vacationdf[(vacationdf["Max Temp"] >= 70) & (vacationdf['Max Temp'] <= 80) & (vacationdf['WindSpeed'] < 10) &
                   (vacationdf['Cloudiness'] < 30) & (vacationdf['Humidity'] < 50) ]
ideal_df



,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,WindSpeed,Country,Date
39,39,Jamestown,42.10,-79.24,73.400,40,1,6.20,US,1591478476
132,132,Jamestown,42.10,-79.24,73.400,40,1,6.20,US,1591478476
247,247,Walvis Bay,-22.96,14.51,75.200,29,0,5.70,NaN,1591478983
305,305,Saint Anthony,45.02,-93.22,78.008,36,1,4.60,US,1591478702
306,306,San Jose,37.34,-121.89,73.994,33,20,9.80,US,1591478945
316,316,Kutum,14.20,24.67,74.390,20,1,2.27,SD,1591478999
362,362,Laguna,38.42,-121.42,78.008,31,1,7.70,US,1591479010
391,391,Isfahan,32.66,51.68,73.400,16,0,2.10,IR,1591479017
416,416,Tāybād,34.74,60.78,76.694,38,2,7.15,IR,1591479022
477,477,Cairo,30.06,31.25,78.800,41,0,5.10,EG,1591478906


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [42]:
ideal_df.loc[:,"Hotel Name"]=""
ideal_df

C:\Users\margarita.garza\Anaconda3\New folder\envs\DataVirtual\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,WindSpeed,Country,Date,Hotel Name
39,39,Jamestown,42.10,-79.24,73.400,40,1,6.20,US,1591478476,
132,132,Jamestown,42.10,-79.24,73.400,40,1,6.20,US,1591478476,
247,247,Walvis Bay,-22.96,14.51,75.200,29,0,5.70,NaN,1591478983,
305,305,Saint Anthony,45.02,-93.22,78.008,36,1,4.60,US,1591478702,
306,306,San Jose,37.34,-121.89,73.994,33,20,9.80,US,1591478945,
316,316,Kutum,14.20,24.67,74.390,20,1,2.27,SD,1591478999,
362,362,Laguna,38.42,-121.42,78.008,31,1,7.70,US,1591479010,
391,391,Isfahan,32.66,51.68,73.400,16,0,2.10,IR,1591479017,
416,416,Tāybād,34.74,60.78,76.694,38,2,7.15,IR,1591479022,
477,477,Cairo,30.06,31.25,78.800,41,0,5.10,EG,1591478906,


In [52]:
hotel_df=[]


In [69]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in ideal_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    #print(json.dumps(name_address, indent=4, sort_keys=True))
    

    #Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        ideal_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        hotel_df.append(name_address["results"][0]["name"])
        print(f"Hotel found nearby. {index}")
    except (KeyError, IndexError):
        print(f"Missing field/result... skipping. {index}")

C:\Users\margarita.garza\Anaconda3\New folder\envs\DataVirtual\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Hotel found nearby. 39
Hotel found nearby. 132
Hotel found nearby. 247
Hotel found nearby. 305
Hotel found nearby. 306
Hotel found nearby. 362
Hotel found nearby. 391
Hotel found nearby. 416
Hotel found nearby. 477
Hotel found nearby. 534
Hotel found nearby. 615


In [70]:
ideal_df


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,WindSpeed,Country,Date,Hotel Name
39,39,Jamestown,42.10,-79.24,73.400,40,1,6.20,US,1591478476,Chautauqua Harbor Hotel
132,132,Jamestown,42.10,-79.24,73.400,40,1,6.20,US,1591478476,Chautauqua Harbor Hotel
247,247,Walvis Bay,-22.96,14.51,75.200,29,0,5.70,NaN,1591478983,Protea Hotel by Marriott Walvis Bay Pelican Bay
305,305,Saint Anthony,45.02,-93.22,78.008,36,1,4.60,US,1591478702,LivINN Hotel Minneapolis North/Fridley
306,306,San Jose,37.34,-121.89,73.994,33,20,9.80,US,1591478945,DoubleTree by Hilton Hotel San Jose
362,362,Laguna,38.42,-121.42,78.008,31,1,7.70,US,1591479010,Hilton Garden Inn Sacramento Elk Grove
391,391,Isfahan,32.66,51.68,73.400,16,0,2.10,IR,1591479017,Abbasi Hotel
416,416,Tāybād,34.74,60.78,76.694,38,2,7.15,IR,1591479022,سه طبقه خیرخواه
477,477,Cairo,30.06,31.25,78.800,41,0,5.10,EG,1591478906,Valencia Hotel
534,534,Griffith,41.53,-87.42,75.002,49,1,4.79,US,1591478919,Courtyard by Marriott Chicago Southeast/Hammon...


In [71]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_df.iterrows()]
locations = ideal_df[["Lat", "Lng"]]

In [78]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)


# Add the layer to the map
fig.add_layer(markers)

fig


Figure(layout=FigureLayout(height='420px'))